In [1]:
#!/usr/bin/env python3
"""open_cv.ipynb
James Gardner 2019

various basic python3-openCV things
"""

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [2]:
def zoom(frame,r=0.5):
    rheight = int(frame.shape[0]*r)
    rwidth  = int(frame.shape[1]*r)
    # caution: order of width, height in resize call
    return cv2.resize(frame,(rwidth,rheight))

In [104]:
# https://www.niser.ac.in/sps/sites/default/files/basic_page/Michelson%20Interferometer_P744%20-%20Optics.pdf
# maximum when 2*d*cos(theta) = n*lambda
# cos[theta] = n*(lambda/(2*d[t]))
# phys_arg is ultimately time varying, lambda/(2*d[t])
phys_arg = 0.019
total_rings = int(1/phys_arg)
angles = np.arccos([n*phys_arg for n in range(1,total_rings+1)])
# print(angles*180/np.pi)

# pretend that source is 1 away from centre of screen of height 2
# tan(theta) gives then gives the fractional radius
frac_radii = np.tan(angles)
frac_radii = frac_radii[frac_radii < 1]
# print(frac_radii)

In [110]:
# write video of rings
fps = 30
duration = 5
total_frames = 30*5
# caution: array[1], array[0]
frame_size = 512, 512
frame_array_size = 512, 512, 3

out = cv2.VideoWriter('green_rings.mp4',cv2.VideoWriter_fourcc(*'mp4v'),fps,frame_size)

for i in tqdm(range(total_frames)):
    out_frame = np.full(frame_array_size, 255).astype('uint8')
    #out_frame[:,i] = [255,255,255]
    #out_frame = np.uint8(cv2.add(frameA,frameB))

    # cv.Circle(img, center, radius, color, thickness=1) 
    circle_pos_height = int(frame_size[1]/2)
    circle_pos_width  = int(frame_size[0]/2)
    min_frame_dim = int(np.min(frame_size)/2)
    oscillation_amp = 5
    oscillation_speed = i/(2*np.pi)
    
    for r in frac_radii:
        circle_radius = int(r*min_frame_dim + oscillation_amp*np.sin(oscillation_speed))
        cv2.circle(out_frame,(circle_pos_width,circle_pos_height),circle_radius,[0,255,0]) 
    
    #cv2.imshow('title',out_frame)
    #if cv2.waitKey(30) & 0xFF == ord('q'):
    #    break
    
    out.write(out_frame)

out.release()
cv2.destroyAllWindows()

In [8]:
cap = cv2.VideoCapture('tracking_test.mp4')
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
pbar = tqdm(total=total_frames)

ret, frame = cap.read()

fps = cap.get(cv2.CAP_PROP_FPS)
# width, height
frame_size = frame.shape[0:2][::-1]

out = cv2.VideoWriter('noisy_tracking_test.mp4',cv2.VideoWriter_fourcc(*'mp4v'),fps,frame_size)

while ret:
    
    #out_frame = cv2.flip(frame,1)
    noise_level = 10
    noisy_frame = np.random.normal(0,noise_level,frame.shape).astype('uint8')
    out_frame = cv2.add(frame,noisy_frame)
    #out_frame = cv2.add(frame,frame*noisy_frame)
    #cv2.normalize(noisy_frame, noisy_frame, 0, 255, cv2.NORM_MINMAX, dtype=-1)
    #noisy_image = noisy_image.astype(np.uint8)
    #out_frame[out_frame < 0] = 0
    #out_frame[out_frame > 255] = 255
    out.write(np.uint8(out_frame))
    
    #cv2.imshow('noisy vid',zoom(out_frame))
    #if cv2.waitKey(30) & 0xFF == ord('q'):
    #    break    
    
    ret, frame = cap.read()
    pbar.update(1)

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture('tracking_test.mp4')
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0
pbar = tqdm(total=total_frames)

ret, frame = cap.read()

while ret:
    cv2.imshow('a',zoom(frame[:,:,1]))

    # show frame for 30 ms 
    # 0xFF necessary on 64-bit to truncate waitKey, & is bit-wise
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break    
    
    ret, frame = cap.read()
    frame_count += 1
    pbar.update(1)

pbar.close()
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture('tracking_test.mp4')
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
pbar = tqdm(total=total_frames)

ret, frame = cap.read()

fps = cap.get(cv2.CAP_PROP_FPS)
# width, height
frame_size = frame.shape[0:2][::-1]

out = cv2.VideoWriter('noisy_tracking_test.mp4',cv2.VideoWriter_fourcc(*'mp4v'),fps,frame_size)

while ret:
    
    #out_frame = cv2.flip(frame,1)
    noise_level = 10
    noisy_frame = np.random.normal(0,noise_level,frame.shape).astype('uint8')
    out_frame = cv2.add(frame,noisy_frame)
    #out_frame = cv2.add(frame,frame*noisy_frame)
    #cv2.normalize(noisy_frame, noisy_frame, 0, 255, cv2.NORM_MINMAX, dtype=-1)
    #noisy_image = noisy_image.astype(np.uint8)
    #out_frame[out_frame < 0] = 0
    #out_frame[out_frame > 255] = 255
    out.write(np.uint8(out_frame))
    
    #cv2.imshow('noisy vid',zoom(out_frame))
    #if cv2.waitKey(30) & 0xFF == ord('q'):
    #    break    
    
    ret, frame = cap.read()
    pbar.update(1)

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# openCV image tracking tutorial
# https://opencv-python-tutroals.readthedocs.io/en/latest/
# py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html
# https://stackoverqflow.com/questions/43063320/cv2-calcopticalflowpyrlk-adding-new-points

cap = cv2.VideoCapture('noisy_tracking_test.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
pbar = tqdm(total=total_frames-1)

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# # params for ShiTomasi corner detection,
# feature_params = dict( maxCorners = 100,
#                        qualityLevel = 0.3,
#                        minDistance = 7,
#                        blockSize = 7 )
# p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# use pinta to get values for selected points
p0 = np.array([[1093,548],[1226,544],[1344,547],[1439,553]],np.float32)
fav_p = []
# print(p0)
# 427 175 , 401 187
# p0 = np.array([[[427,175]],[[401,187]]])

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
ret, frame = cap.read()

after_first = False

while ret:
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    #print(p1,p0,st,err)
    good_new = p1#[st==1]
    good_old = p0#[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',zoom(img))
    if cv2.waitKey(15) & 0xff == ord('q'):
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    pbar.update(1)
    
    if after_first:
        fav_p.append([p[0][0] for p in p1])    
    else:
        after_first = True
    
    ret,frame = cap.read()

fav_p = np.array(fav_p)
pbar.close()    
cv2.destroyAllWindows()
cap.release()

In [ ]:
# implementation from tracker_time_series
# after_first loses a frame
signal_frames = total_frames - 2
total_time = (signal_frames-2)/fps
signal = fav_p[:,0]
t = np.linspace(0,total_time,signal_frames)
dt = t[1] - t[0]

yf = np.fft.fft(signal)
# normalised-absolute value of FT'd signal
nrm_abs_yf = 2/signal_frames*np.abs(yf)
freq_scale = np.fft.fftfreq(len(yf),dt)
# np.fft.fftfreq outputs 0 to +inf then -inf to 0, so :N//2 gets +ve side; wild!
freq_prob = nrm_abs_yf[:signal_frames//2]

fig, (ax0,ax1) = plt.subplots(2,figsize=(14,14))
ax0.plot(t,signal)
ax1.plot(freq_scale[:signal_frames//2],freq_prob)
ax0.set(title='noisy tracked signal',ylabel='horizonal position, x / frames',xlabel='time, t / s')
ax1.set(title='discrete FFT',ylabel='freq strength in signal',xlabel='frequency, f / Hz')
plt.savefig('noisy_tracker_signal.pdf',bbox_inches='tight')
plt.clf()

In [ ]:
# img = cv2.imread()
cap = cv2.VideoCapture('tracking_test.mp4')
ret, frame = cap.read()
g_frame = frame[:,:,1]
# print(g_frame.shape)

# cv2.namedWindow('a',cv2.WINDOW_NORMAL)
# cv2.namedWindow('a',cv2.WINDOW_AUTOSIZE) 
cv2.imshow('a',zoom(g_frame))
cv2.waitKey(0)
cv2.destroyAllWindows()
cap.release()

In [ ]:
cap = cv2.VideoCapture('tracking_test.mp4')
# ret is 'did it successfully return another frame?'

while True:
    if cv2.waitKey(1) == ord('q'):
        break
    
    ret, frame = cap.read()
    
    try:
        grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('frame',frame)
        cv2.imshow('greyframe',grey)
    except:
        cap.release()
        cv2.destroyAllWindows()        
        break    

cap.release()
cv2.destroyAllWindows()

In [ ]:
img = cv2.imread('bgr_tester.jpeg')
# "It is important to note that OpenCV reads colors as BGR (Blue Green Red),
# where most computer applications read as RGB (Red Green Blue).
# Remember this!"
b_channel = img[:,:,0]
g_channel = img[:,:,1]
r_channel = img[:,:,2]

cv2.imshow('b',b_channel)
cv2.imshow('g',g_channel)
cv2.imshow('r',r_channel)
# waitKey argument is the number of milliseconds it waits for
if cv2.waitKey(0) == ord('q'):
    # cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# frame splitter!
cap = cv2.VideoCapture('tracking_test.mp4')
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count = 0
pbar = tqdm(total=total_frames)

ret = True
while ret:    
    ret, frame = cap.read()
    if not ret:
        break
        
    #cv2.imwrite('./plots/tracking_test_{:03d}.jpeg'.format(frame_count),frame)
    
    frame_count += 1
    pbar.update(1)
    
pbar.close()
cap.release()
cv2.destroyAllWindows()